In [1]:
######################## Standardizing the water and ions for various protein conformations ############################
## -> Solved
## We now have a set of functions that can model any conformation against a model conformation. The model conformation is the solvated conformation. The functions will copy the protein residues and their positions, then add the exact amount of water and ion atoms as the model, while checking and avoiding steric clashes. After this, the new solvated protein can be minimized and equilibrated, and then used for the production run.
# One thing which I am not sure where it should go is the fixing of naked charges:
# We can follow the fixing of naked charges for the model conformation, and thus use the same system and forcefields for all the conformations. This will make sure that the charges are fixed in the same way for all the conformations.
# We can also fix the naked charges for each conformation separately, but then we will have a new system (and forcefields) for each conformation. 

# For now, we will fix the naked charges for the model conformation, and then use the same system and forcefields for all the conformations. This makes for a simpler implementation, and will make sure that the charges are fixed in the same way for all the conformations.
# An important question to consider is the following: Will every conformation need its naked charges to be fixed? Or are these naked charges the same for all conformations? Intuitevely, I would say that the naked charges are the same for all conformations. The reason is that the same fixings are used for a proetins that is modifying conformations in a simulation, regardless of the conformation. However, I will have to check this to make sure.

## Problem
# TODO: We will have to minimize and equilibrate all the conformaitons before we run the simulation in production mode

# Old system is the system of the wt, and any of its derivatives
# New system is the system of the mutant, and any of its derivatives

# It might be that every conformation will need its own system, so we will have to create a new system for each conformation.
# up till now it does not seem that we will have to create a new system for each conformation, but we will have to create a new system for each mutant, and then a hybrid system

# For now, make sure that you are keeping the same system for all the conformations of the same protein (wt or mutant)


# TODO: We have to make sure that the pdbs all have the same nomeclature convention, specifically the IUPAC naming convention. This should work fine now, as long as the atoms namings into the pipeline follow the IUPAC naming convention.
## In perses, the residues used follow an older PDB bank format, however, the proteins it outputs follow the new IUPAC naming convention.
## Moreover, we have to make sure that the order of atoms is the same in all the pdbs, this is importnat to make sure that the atom indices are the same in all the pdbs.
#I have implemented a function that can rewrite a pdb to the IUPAC naming convention, along with best-practice ordering of atoms. It rearranges the order of the atoms in the pdb to the best-practice ordering, and renames the atoms to the IUPAC naming convention. I also updateds any CONECT records in the pdb to reflect the new atom names.


################### Problem of the ordering of the atoms in the pdb ############################
### -> Fixed at two levels:
## 1. the perses/rjmc/topology_proposal.py _add_new_atoms now checks for order according to a dictionary holding the ordering of atoms for each residue according to IUPAC naming convention.
## 2. we have a function that can reorder the atoms in the pdb according to best practices. For the function to work, the pdb must be in the IUPAC naming convention. The function will reorder the atoms in the pdb according to the best practices, and will also update the CONECT records in the pdb to reflect the new atom indixes.

##Original Problem::
## One thing I noticed was that the ordering of thea atoms in the new residue of the mutated pdb is different from the ordering of convention. I wanted to make sure that this has nothing to do with our implementation of the RDKIT instead of the openeye toolkit. I will investigate where the difference is originating from. For now, it seems that the difference is originating from the perses mutation toolkit, and is not related to our swapping of the openeye toolkit with RDKIT. I will have to investigate this further.
# This will be a problem when we create our own conformations, as the orderings of the atoms will be different. Indices in the different maps will be different, and we will have to deal with this problem. 



##Problem: We have to create a hybrid topology for each conformation.
# Another problem is that we will have to create a hybrid topology and position for every conformation pair. We will have to add the positions of the mutant atoms in a way that woul make physical sense. We have to find a way to implement this as well.



# TODO: We have to add a function that checks the position of the alchemical ion for each conformation, and then make sure it is outside the protein radius up to a certain cutoff.


In [1]:
import openmm
import openmm.app as app
import openmm.unit as unit
from openmm.unit import nanometer, molar, elementary_charge, norm
import mdtraj as md
from openmm.app import PDBFile, Modeller, ForceField, Simulation, Topology
import copy
import openmm.app.element as elem
import random
from math import floor
import pickle

In [2]:
pickle_file= "/Users/Apple/Desktop/MD_Simulations/openmm_for perses/data/output_apo.pickle"

with open(pickle_file, 'rb') as f:
    loaded_pickle = pickle.load(f)

htf variable:
_default_electrostatics_expression_list : list of str
        strings that will form the custom expression that will be used to create the CustomNonbondedForce for electrostatics
    _default_sterics_expression_list : list of str
        strings that will form the custom expression that will be used to create the CustomNonbondedForce for sterics
    _default_exceptions_expression_list : list of str
        strings that will form the custom expression that will be used to create the CustomBondForce for exceptions
    _default_electrostatics_expression : str
        the custom expression that will be used to create the CustomNonbondedForce for electrostatics (created from joining the corresponding list of strings)
    _default_sterics_expression : str
        the custom expression that will be used to create the CustomNonbondedForce for sterics (created from joining the corresponding list of strings)
    _default_exceptions_expression : str
        the custom expression that will be used to create the CustomBondForce for exceptions (created from joining the corresponding list of strings)
    _topology_proposal : perses.rjmc.topology_proposal.TopologyProposal
        topology proposal of the old-to-new transformation
    _old_system : openmm.System
        the old system
    _new_system : openmm.System
        the new system
    _old_to_hybrid_map : dict of ints
        key: index of atom in old system, value: index of corresponding atom in the hybrid system
    _new_to_hybrid_map : dict of ints
        key: index of atom in new system, value: index of corresponding atom in the hybrid system
    _hybrid_system_forces : dict of key: str, value: openmm custom force objects
        key : name of the custom force (e.g. CustomBondForce, CustomNonbondedForce_sterics), value : custom force object
    _old_positions : [n,3] np.ndarray of float
        positions of coordinates of old system, where n is the number of atoms in the old system
    _new_positions : [m,3] np.ndarray of float
        positions of coordinates of new system, where m is the number of atoms in the new system
    _old_system_forces : dict of key: str, value: openmm force object
        key: name of the force (e.g. HarmonicBondForce), value: force object in the old system
    _new_system_forces : dict of key: str, value: openmm force object
        key: name of the force (e.g. HarmonicBondForce), value: force object in the new system
    _nonbonded_method : openmm.NonbondedForce.NonbondedMethod
        nonbonded method to use for the hybrid system, determined from the old system's nonbonded method
    _cutoff : openmm.unit.nanometer
        cutoff distance to be used in for the forces that encompass the nonbonded interactions in the hybrid system
    _alpha_ewald : float
        alpha to be used for PME electrostatics in the CustomNonbondedForce_electrostatics and CustomBondForce_exceptions forces
    _w_lifting : float
        maximal length of the 4th dimension
    _hybrid_system : openmm.System
        the hybrid system that allows both alchemical and rest scaling
    _hybrid_to_old_map : dict of ints
        key: index of atom in hybrid system, value: index of corresponding atom in old system
        aka _old_to_hybrid_map with keys and values flipped
    _hybrid_to_new_map : dict of ints
        key: index of atom in hybrid system, value: index of corresponding atom in new system
        aka _new_to_hybrid_map with keys and values flipped
    _atom_classes : dict of key: string, value: set
        key: name of the atom class (unique_old_atoms, unique_new_atoms, core_atoms, environment_atoms)
        value: set of (hybrid-indexed) atoms in the atom class
    _hybrid_positions : [l,3] np.ndarray of float
        positions of coordinates of hybrid system, where l is the number of atoms in the hybrid system
    _hybrid_topology : mdtraj.Topology
        topology for the hybrid system
    _rest_radius : float
        radius for the rest region, in nanometers
    _rest_region : list of int
        list of (hybrid-indexed) atoms that should be considered as part of the rest region
    _atom_idx_to_object : dict of key: int, value: openmm or mtraj atom object
        key: hybrid system index for the atom, value: openmm.app.topology.Atom or mdtraj.topology.Atom object (depending on what class type self._hybrid_topology is)
    _old_system_exceptions : dict of key: tuple of ints, value: list of floats
        key: old system indices of the atoms in the exception, value: chargeProd (units of the proton charge squared), sigma (nm), and epsilon (kJ/mol) for the exception
    _new_system_exceptions : dict of key: tuple of ints, value: list of floats
        key: new system indices of the atoms in the exception, value: chargeProd (units of the proton charge squared), sigma (nm), and epsilon (kJ/mol) for the exception


_topology_proposal:


self._new_topology = new_topology
self._new_system = new_system
self._old_topology = old_topology
self._old_system = old_system
self._logp_proposal = logp_proposal
self._new_chemical_state_key = new_chemical_state_key
self._old_chemical_state_key = old_chemical_state_key
self._old_residue_name = old_residue_name
self._new_residue_name = new_residue_name
self._new_to_old_atom_map = new_to_old_atom_map
self._old_to_new_atom_map = {old_atom : new_atom for new_atom, old_atom in new_to_old_atom_map.items()}
self._unique_new_atoms = list(set(range(self._new_topology.getNumAtoms()))-set(self._new_to_old_atom_map.keys()))
self._unique_old_atoms = list(set(range(self._old_topology.getNumAtoms()))-set(self._new_to_old_atom_map.values()))
self._old_alchemical_atoms = set(old_alchemical_atoms) if (old_alchemical_atoms is not None) else {atom for atom in range(old_system.getNumParticles())}
self._new_alchemical_atoms = set([self._old_to_new_atom_map[old_alch_atom] for old_alch_atom in self._old_alchemical_atoms if old_alch_atom in list(self._new_to_old_atom_map.values())]).union(set(self._unique_new_atoms))
self._old_environment_atoms = set(range(old_system.getNumParticles())) - self._old_alchemical_atoms
self._new_environment_atoms = set(range(new_system.getNumParticles())) - self._new_alchemical_atoms
self._metadata = metadata
self._core_new_to_old_atom_map = {new_atom: old_atom for new_atom, old_atom in self._new_to_old_atom_map.items() if new_atom in self._new_alchemical_atoms and old_atom in self._old_alchemical_atoms}


### Notes:

- Number of atoms should be identical for all except the atoms involved in the mutated residue
- New and old positions seem to be identical for mapped atoms. Only the unique old and new atoms are varying 
- Atoms other than the unique old and new atoms are identical in the hybrid topology as well

--> It seems that it would be almost impossible to have starting ending positions that are not identical for the non-unique atoms

In [3]:
wt_topology = loaded_pickle._topology_proposal.old_topology # openmm topology object
wt_positions = loaded_pickle._old_positions # numpy array of 3D positions in nm (shape=(n_atoms, 3))  openmm.unit.quantity.Quantity

hybrid_topology = loaded_pickle._hybrid_topology # mdtraj topology object
system = loaded_pickle._old_system # openmm system object
periodic_box_vectors = system.getDefaultPeriodicBoxVectors() # openmm.unit.quantity.Quantity
print(type(system))
print(periodic_box_vectors)

<class 'openmm.openmm.System'>
[Quantity(value=Vec3(x=6.428100000000001, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=6.428100000000001, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=6.428100000000001), unit=nanometer)]


In [4]:
# now we have pdb apo-wt-1.pdb

# load the pdb file
pdb_wt_p_pdb = PDBFile("/Users/Apple/Desktop/MD_Simulations/openmm_for perses/data/apo-wt-1.pdb")
protein_wt_p_positions, protein_wt_p_topology, protein_wt_p_md_topology = pdb_wt_p_pdb.positions, pdb_wt_p_pdb.topology, md.Topology.from_openmm(pdb_wt_p_pdb.topology)
# protein_wt_p_positions is an openmm topology object in nm
# protein_wt_p_topology is an openmm topology object

print(type(protein_wt_p_positions))



<class 'openmm.unit.quantity.Quantity'>


In [100]:
# get the number of protein residues in the wt_p system

non_protein_residues = ['HOH', 'WAT', 'NA', 'CL']

# Count the number of protein residues
protein_residue_count_wt = sum(1 for res in wt_topology.residues() if res.name not in non_protein_residues)

print(f"Number of protein residues: {protein_residue_count_wt}")

# get the number of protein residues in the wt_p system

# Count the number of protein residues
protein_residue_count_wt_p = sum(1 for res in protein_wt_p_topology.residues() if res.name not in non_protein_residues)


print(f"Number of protein residues: {protein_residue_count_wt_p}")


# get the number of protein atoms in the wt_p system

# Count the number of protein atoms
protein_atom_count_wt_p = sum(1 for atom in protein_wt_p_topology.atoms() if atom.residue.name not in non_protein_residues)

print(f"Number of protein atoms in wt_p: {protein_atom_count_wt_p}")

# get the number of protein atoms in the wt system

# Count the number of protein atoms
protein_atom_count_wt = sum(1 for atom in wt_topology.atoms() if atom.residue.name not in non_protein_residues)

print(f"Number of protein atoms in wt: {protein_atom_count_wt}")


#get the number of water atoms in the wt_p system

water_residues = ['HOH', 'WAT']

# Count the number of water residues
water_count_wt_p = sum(1 for res in protein_wt_p_topology.residues() if res.name in water_residues)

print(f"Number of water residues in wt_p: {water_count_wt_p}")

# get the number of water atoms in the wt_p system

# Count the number of water residues
water_count_wt = sum(1 for res in wt_topology.residues() if res.name in water_residues)

print(f"Number of water residues in wt: {water_count_wt}")

# get the number of ions in the wt_p system

ion_residues = ['NA', 'CL']

# Count the number of ion residues
ion_count_wt_p = sum(1 for res in protein_wt_p_topology.residues() if res.name in ion_residues)

print(f"Number of ion residues in wt_p: {ion_count_wt_p}")

# get the number of ions in the wt system

# Count the number of ion residues
ion_count_wt = sum(1 for res in wt_topology.residues() if res.name in ion_residues)

print(f"Number of ion residues in wt: {ion_count_wt}")

# total is 24,969 atoms in the wt_p system

Number of protein residues: 110
Number of protein residues: 110
Number of protein atoms in wt_p: 1709
Number of protein atoms in wt: 1709
Number of water residues in wt_p: 0
Number of water residues in wt: 7748
Number of ion residues in wt_p: 0
Number of ion residues in wt: 16


In [66]:
def check_topology_alignment(topology_a, topology_b):
    # function that makes sure the two topologies have the same atom names and order for the protein residues

    # Get the protein residues from the two topologies
    protein_a = [residue for residue in topology_a.residues() if residue.name in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']]

    protein_b = [residue for residue in topology_b.residues() if residue.name in ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']]

    # Check if the two topologies have the same number of protein residues
    if len(protein_a) != len(protein_b):
        raise ValueError(f'The two topologies do not have the same number of protein residues: \n {len(protein_a)} protein residues in topology A and {len(protein_b)} protein residues in topology B')
    
    # check if the two topologies have the same order of protein residues
    for residue_a, residue_b in zip(protein_a, protein_b):
        if residue_a.name != residue_b.name:
            raise ValueError(f'The two topologies do not have the same order of protein residues: \n {residue_a.name} in topology A and {residue_b.name} in topology B')
    
    # Check if the two topologies have the same number of atoms in each protein residue, and return the residue for which the number of atoms is different if it exists
    for residue_a, residue_b in zip(protein_a, protein_b):
        if len([residue_a.atoms()]) != len([residue_b.atoms()]):
            raise ValueError(f'The two topologies do not have the same number of atoms in the following protein residues: \n {residue_a.name} in topology A has {len(residue_a.atoms())} atoms and {residue_b.name} in topology B has {len(residue_b.atoms())} atoms')

            
    
    #check if the order of atoms in each protein residue is the same
    for residue_a, residue_b in zip(protein_a, protein_b):
        for atom_a, atom_b in zip(residue_a.atoms(), residue_b.atoms()):
            if atom_a.name != atom_b.name:
                raise ValueError(f'The two topologies do not have the same order of atoms in the following protein residues: \n {atom_a.name} in topology A and {atom_b.name} in topology B')
                
    print('The two topologies have the same atom names and order for the protein residues')
    

In [286]:
#def solvate_with_conformation_a(prtn_a_topology: openmm.app.topology.Topology, 
                                #prtn_a_positions: openmm.unit.quantity.Quantity, 
                                prtn_b_topology:openmm.app.topology.Topology,
                                prtn_b_positions:openmm.unit.quantity.Quantity, 
                                boxVectors, 
                                padding=1.1 * unit.nanometers):
    """
    Solvate conformation B using water and ions from conformation A.
    We do the following:
    1. Load conformation A and B
    2. Count the number of water, Na, and Cl in conformation A
    3. Add a number of water molecules to conformation B that is equal to the number of (water molecules + Na + Cl) in conformation A. We do so because we cannot specify the number of ions to add directly.
    4. Randomly replace water molecules with Na and Cl ions, to the exact number of Na and Cl ions in conformation A. Instead of replacing the randomly selected water molecules, we will remove them and add Na and Cl ions at the same positions, at the end of the PDB file, to a new chain.
    pdb_a: str
        Path to the PDB file of conformation a that is solvated.
    pdb_b: str
        Path to the PDB file of conformation b that is solvated.
    output_solvated_pdb_b: str
        Path and file name to save the solvated conformation B.
    """
    import random
    from openmm.app.element import Element
    from openmmforcefields.generators import SystemGenerator
    from openmm.openmm import NonbondedForce

    
    
    
    

    
    forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
    forcefield_kwargs = {'removeCMMotion': False, 'constraints' : app.HBonds, 'rigidWater': True, 'hydrogenMass' : 3 * unit.amus}
    periodic_forcefield_kwargs = {'nonbondedMethod': app.PME, 'ewaldErrorTolerance': 0.00025}
    nonperiodic_forcefield_kwargs = None
    system_generator = SystemGenerator(forcefields=forcefield_files,
                                                    barostat=barostat,
                                                    forcefield_kwargs=forcefield_kwargs,
                                                    periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                    nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs)
    water_model = 'tip3p'
    
    # Count the number of water, Na, and Cl in conformation A
    
    water_count = sum(1 for res in prtn_a_topology.residues() if res.name == 'HOH')
    na_count = sum(1 for res in prtn_a_topology.residues() if res.name == 'NA')
    cl_count = sum(1 for res in prtn_a_topology.residues() if res.name == 'CL')

    print(f"Water count in A: {water_count}, Na count in A: {na_count}, Cl count in A: {cl_count}")

    # Total number of solvent molecules to add
    total_solvent_count = water_count + na_count + cl_count

    # Create a Modeller object for conformation B
    modeller_b = Modeller(prtn_b_topology, prtn_b_positions)

    # get the number of Na and Cl ions in the solvated conformation B
    na_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'NA')
    cl_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'CL')

    print(f"Original Na count in B: {na_count_b} and Original Cl count in B: {cl_count_b}")

    # Add solvent (water) to conformation B
    modeller_b.addSolvent(system_generator.forcefield,
                          model=water_model,
                          numAdded=total_solvent_count,
                          #boxVectors=boxVectors, 
                          neutralize= False)
    # get the number of water molecules added
    water_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'HOH')
    print(f"Water count in B: {water_count_b} and total solvent count: {total_solvent_count}")
    
    # Remove water molecules added to conformation B that are in excess of the total_solvent_count
    if water_count_b > total_solvent_count:
        print("Removing water molecules from B")
        # Randomly select water molecules to remove
        water_residues = [res for res in modeller_b.topology.residues() if res.name == 'HOH']
        random.shuffle(water_residues)
        # Remove the excess water molecules
        for i in range(water_count_b - total_solvent_count):
            water_atoms = list(water_residues[i].atoms())
            modeller_b.delete(water_atoms)
    
    elif water_count_b < total_solvent_count:
        print("Adding water molecules to B")
        ## Add water molecules to conformation B that are in deficit of the total_solvent_count
        modeller_b.addSolvent(system_generator.forcefield,
                              model=water_model,
                              numAdded=total_solvent_count - water_count_b, 
                              neutralize= False)
        

    # check the number of water molecules in the final solvated conformation B
    water_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'HOH')
    print(f"Water count in B after adding or removing: {water_count_b} and total solvent count: {total_solvent_count}")

    # get the total number of atoms in modeller_b
    total_atoms = sum(1 for atom in modeller_b.topology.atoms())
    print(f"Total atoms in modeller_b: {total_atoms}")

    # get the number of Na and Cl ions in the solvated conformation B
    na_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'NA')
    cl_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'CL')

    print(f"Na count in B: {na_count_b} and Cl count in B: {cl_count_b}")
    

    # Randomly replace water molecules with Na and Cl ions
    residues = list(modeller_b.topology.residues())
    water_residues = [res for res in residues if res.name == 'HOH']


     # Randomly select water molecules to convert to Na and Cl
    random.shuffle(water_residues)
    positions = modeller_b.positions

    # Store positions for new ions
    new_positions = []
    residues_to_delete = []

    # Convert water to Na
    for i in range(na_count):
        water_residue = water_residues[i]
        water_atoms = list(water_residue.atoms())
        # Store the position of the oxygen atom
        new_positions.append(positions[water_atoms[0].index])
        # water molecule to remove
        print(f"{water_residue} will be deleted")
        residues_to_delete.append(water_residue)

    # Convert water to Cl
    for i in range(na_count, na_count+ cl_count):
        water_residue = water_residues[i]
        water_atoms = list(water_residue.atoms())
        # Store the position of the oxygen atom
        new_positions.append(positions[water_atoms[0].index])
        # water molecule to remove
        print(f"{water_residue} will be deleted")
        
        residues_to_delete.append(water_residue)
    
    print(f"Residues to delete: {residues_to_delete}")
    # Delete the water residues
    modeller_b.delete(residues_to_delete)

    # get the total number of water molecules in the solvated conformation B after removing some
    water_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'HOH')
    print(f"Water count in B after removing some: {water_count_b}")
    
    
    # Add a single chain for all ions
    ion_chain = modeller_b.topology.addChain()
    ion_topology = Topology()
    
    print(f"Length of new_positions: {len(new_positions)}")

    print("#######Positions########")
    print(new_positions)

    # Add Na ions
    for pos in new_positions[:na_count]:
        
        na_residue = ion_topology.addResidue('NA', ion_chain)
        ion_topology.addAtom('NA', Element.getBySymbol('Na'), na_residue)
              

    # Add Cl ions
    for pos in new_positions[na_count:]:
        cl_residue = ion_topology.addResidue('CL', ion_chain)
        ion_topology.addAtom('CL', Element.getBySymbol('Cl'), cl_residue)

    # print all the ions in the ion topology
    print("Ions in the ion topology")
    ions = list(ion_topology.residues())
    print(ions)
    print(f"number of atoms in the ion topology: {sum(1 for atom in ion_topology.residues())}")
    
    
    # print all the positions of the ions
    for pos in new_positions:
        print("--")
        print(pos)
        
    
    # add the topology, and the positions of the ions to the modeller object
    modeller_b.add(ion_topology, new_positions)


    # get the total number of atoms in modeller_b
    total_atoms = sum(1 for atom in modeller_b.topology.atoms())
    print(f"Total atoms in modeller_b after adding ions: {total_atoms}")


    # get the number of Na and Cl ions in the solvated conformation B
    na_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'NA')
    cl_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'CL')

    print(f"Na count in B: {na_count_b} and Cl count in B: {cl_count_b}")
    

    # Copy the topology from conformation A and the positions from the solvated conformation B to create the final Modeller object
    modeller_final = Modeller(prtn_a_topology, modeller_b.positions)

    # generate the system
    solvated_system = system_generator.create_system(modeller_final.topology)

    
    # check the charge of the system
    nonbonded_force = [f for f in solvated_system.getForces() if isinstance(f, NonbondedForce)][0]
    total_charge = sum(nonbonded_force.getParticleParameters(i)[0].value_in_unit(unit.elementary_charge) for i in range(nonbonded_force.getNumParticles()))
    print(f"Total charge of the system: {total_charge}")    

    

    # Fix naked charges
    atom_map = {atom.index: atom.residue.name for atom in modeller_final.topology.atoms()}
    force_dict = {force.__class__.__name__: force for force in solvated_system.getForces()}
    if 'NonbondedForce' in [k for k in force_dict.keys()]:
        nb_force = force_dict['NonbondedForce']
        for idx in range(nb_force.getNumParticles()):
            if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
                continue
            charge, sigma, epsilon = nb_force.getParticleParameters(idx)
            if sigma == 0*unit.nanometer:
                new_sigma = 0.06*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
            if epsilon == 0*unit.kilojoule_per_mole:
                new_epsilon = 0.0001*unit.kilojoule_per_mole
                nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
                print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
                if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                    new_sigma = 0.1*unit.nanometer
                    nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                    print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

    
    # check the charge of the system after fixing naked charges
    nonbonded_force = [f for f in solvated_system.getForces() if isinstance(f, NonbondedForce)][0]
    total_charge = sum(nonbonded_force.getParticleParameters(i)[0].value_in_unit(unit.elementary_charge) for i in range(nonbonded_force.getNumParticles()))
    print(f"Total charge of the system after fixing naked charges: {total_charge}")

    return modeller_final, solvated_system

In [5]:
def addIons(topology,positions, replaceableMols, numPositive:int, numNegative:int, ionCutoff=0.05*nanometer, positiveIon='Na+', negativeIon='Cl-'):
        """Adds ions to the system by replacing certain molecules.

        Parameters
        ----------
        forcefield : ForceField
            the ForceField to use to determine the total charge of the system.
        numWaters : int
            the total number of water molecules in the simulation box, used to
            calculate the number of ions / concentration to add.
        replaceableMols : dict
            the molecules to replace by ions, as a dictionary of residue:positions
        ionCutoff: distance=0.5*nanometer
        positiveIon : string='Na+'
            the type of positive ion to add.  Allowed values are 'Cs+', 'K+', 'Li+', 'Na+', and 'Rb+'
        negativeIon : string='Cl-'
            the type of negative ion to add.  Allowed values are 'Cl-', 'Br-', 'F-', and 'I-'. Be aware
            that not all force fields support all ion types.
        ionicStrength : concentration=0*molar
            the total concentration of ions (both positive and negative) to add.  This
            does not include ions that are added to neutralize the system.
            Note that only monovalent ions are currently supported.
        neutralize : bool=True
            whether to add ions to neutralize the system
        residueTemplates : dict=dict()
            specifies which template the ForceField should use for particular residues.  The keys
            should be Residue objects from the Topology, and the values should be the names of the
            templates to use for them.  This is useful when a ForceField contains multiple templates
            that can match the same residue (e.g Fe2+ and Fe3+ templates in the ForceField for a
            monoatomic iron ion in the Topology).
        """

        posIonElements = {'Cs+': elem.cesium, 'K+': elem.potassium,
                          'Li+': elem.lithium, 'Na+': elem.sodium,
                          'Rb+': elem.rubidium}
        negIonElements = {'Cl-': elem.chlorine, 'Br-': elem.bromine,
                          'F-': elem.fluorine, 'I-': elem.iodine}

        ionPositions = []

        numReplaceableMols = len(replaceableMols)

        # Fetch ion elements from user input
        if positiveIon not in posIonElements:
            raise ValueError('Illegal value for positive ion: {}'.format(positiveIon))
        if negativeIon not in negIonElements:
            raise ValueError('Illegal value for negative ion: {}'.format(negativeIon))
        positiveElement = posIonElements[positiveIon]
        negativeElement = negIonElements[negativeIon]

        totalIons = numPositive + numNegative
        
        if totalIons > 0:
            # Randomly select a set of waters
            # while ensuring ions are not placed too close to each other.
            modeller = Modeller(topology, positions)

            replaceableList = list(replaceableMols.keys())
            numAddedIons = 0
            numTrials = 10  # Attempts to add ions N times before quitting
            toReplace = []  # list of molecules to be replaced
            while numAddedIons < totalIons:
                pickedMol = random.choice(replaceableList)
                print(f"pickedMol: {pickedMol}")
                replaceableList.remove(pickedMol)
                # Check distance to other ions
                for pos in ionPositions:
                    distance = norm(pos - replaceableMols[pickedMol])
                    if distance <= ionCutoff:
                        numTrials -= 1
                        break
                else:
                    toReplace.append(pickedMol)
                    ionPositions.append(replaceableMols[pickedMol])
                    numAddedIons += 1

                    n_trials = 10

                if n_trials == 0:
                    raise ValueError('Could not add more than {} ions to the system'.format(numAddedIons))
            print(f"toReplace: {toReplace}")
            # for each water to replace, try to find it in the modeller object. If not found, raise an error
            # get all residues in the modeller object
            all_residues = list(modeller.topology.residues())
            for water in toReplace:
                print(f"searching for {water}")
                if water not in all_residues:
                    raise ValueError('Could not find water to replace in modeller object')
                else:
                    print(f"found {water}")
            # Replace waters/ions in the topology
            print(f"total number of atoms in modeller object before replacing: {sum(1 for atom in modeller.topology.atoms())}")
            modeller.delete(toReplace)
            print(f"total number of atoms in modeller object after replacing: {sum(1 for atom in modeller.topology.atoms())}")
            ionChain = modeller.topology.addChain()
            for i, water in enumerate(toReplace):
                element = (positiveElement if i < numPositive else negativeElement)
                newResidue = modeller.topology.addResidue(element.symbol.upper(), ionChain)
                modeller.topology.addAtom(element.symbol, element, newResidue)
                modeller.positions.append(replaceableMols[water])

            # Update topology/positions
            return modeller.topology, modeller.positions

In [11]:
def solvate_with_conformation_a(prtn_a_topology: openmm.app.topology.Topology, 
                                #prtn_a_positions: openmm.unit.quantity.Quantity, 
                                prtn_b_topology:openmm.app.topology.Topology,
                                prtn_b_positions:openmm.unit.quantity.Quantity, 
                                boxVectors, 
                                padding=1.1 * unit.nanometers):
    """
    Solvate conformation B using water and ions from conformation A.
    We do the following:
    1. Load conformation A and B
    2. Count the number of water, Na, and Cl in conformation A
    3. Add a number of water molecules to conformation B that is equal to the number of (water molecules + Na + Cl) in conformation A. We do so because we cannot specify the number of ions to add directly.
    4. Randomly replace water molecules with Na and Cl ions, to the exact number of Na and Cl ions in conformation A. Instead of replacing the randomly selected water molecules, we will remove them and add Na and Cl ions at the same positions, at the end of the PDB file, to a new chain.
    pdb_a: str
        Path to the PDB file of conformation a that is solvated.
    pdb_b: str
        Path to the PDB file of conformation b that is solvated.
    output_solvated_pdb_b: str
        Path and file name to save the solvated conformation B.
    """
    import random
    from openmmforcefields.generators import SystemGenerator
    from openmm.openmm import NonbondedForce

    
    
    
    

    
    forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
    barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
    forcefield_kwargs = {'removeCMMotion': False, 'constraints' : app.HBonds, 'rigidWater': True, 'hydrogenMass' : 3 * unit.amus}
    periodic_forcefield_kwargs = {'nonbondedMethod': app.PME, 'ewaldErrorTolerance': 0.00025}
    nonperiodic_forcefield_kwargs = None
    system_generator = SystemGenerator(forcefields=forcefield_files,
                                                    barostat=barostat,
                                                    forcefield_kwargs=forcefield_kwargs,
                                                    periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                    nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs)
    water_model = 'tip3p'
    
    # Count the number of water, Na, and Cl in conformation A
    
    water_count = sum(1 for res in prtn_a_topology.residues() if res.name == 'HOH')
    na_count = sum(1 for res in prtn_a_topology.residues() if res.name == 'NA')
    cl_count = sum(1 for res in prtn_a_topology.residues() if res.name == 'CL')
    prtn_residue_count= sum(1 for res in prtn_a_topology.residues() if res.name not in ['HOH', 'WAT', 'NA', 'CL'])

    print(f"Water count in A: {water_count}, Na count in A: {na_count}, Cl count in A: {cl_count}")
    print(f"Total number of atoms in A: {sum(1 for atom in prtn_a_topology.atoms())}")
    print(f"Total number of protein residues in A: {prtn_residue_count}")

    # Total number of solvent molecules to add
    total_solvent_count = water_count + na_count + cl_count

    # Create a Modeller object for conformation B
    modeller_b = Modeller(prtn_b_topology, prtn_b_positions)

    # get the number of Na and Cl ions in the solvated conformation B
    na_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'NA')
    cl_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'CL')

    print(f"Original Na count in B: {na_count_b} and Original Cl count in B: {cl_count_b}")

    # Add solvent (water) to conformation B
    modeller_b.addSolvent(system_generator.forcefield,
                          model=water_model,
                          #numAdded=total_solvent_count,
                          boxVectors=boxVectors, 
                          neutralize= False)
    # get the number of water molecules added
    water_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'HOH')
    print(f"Water count in B: {water_count_b} and total solvent count: {total_solvent_count}")
    
    # Remove water molecules added to conformation B that are in excess of the total_solvent_count
    if water_count_b > total_solvent_count:
        print("Removing water molecules from B")
        # Randomly select water molecules to remove
        water_residues = [res for res in modeller_b.topology.residues() if res.name == 'HOH']
        random.shuffle(water_residues)
        # Remove the excess water molecules
        for i in range(water_count_b - total_solvent_count):
            water_atoms = list(water_residues[i].atoms())
            modeller_b.delete(water_atoms)
    
    elif water_count_b < total_solvent_count:
        print("Adding water molecules to B")
        ## Add water molecules to conformation B that are in deficit of the total_solvent_count
        modeller_b.addSolvent(system_generator.forcefield,
                              model=water_model,
                              numAdded=total_solvent_count - water_count_b, 
                              neutralize= False)
        

    # check the number of water molecules in the final solvated conformation B
    water_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'HOH')
    print(f"Water count in B after adding or removing: {water_count_b} and total solvent count: {total_solvent_count}")

    # get the total number of atoms in modeller_b
    total_atoms = sum(1 for atom in modeller_b.topology.atoms())
    print(f"Total atoms in modeller_b: {total_atoms}")

    # get the number of Na and Cl ions in the solvated conformation B
    na_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'NA')
    cl_count_b = sum(1 for res in modeller_b.topology.residues() if res.name == 'CL')

    print(f"Na count in B: {na_count_b} and Cl count in B: {cl_count_b}")

    ##get the names of the first 10 protein residues in the solvated conformation B
    protein_residues = [res for res in modeller_b.topology.residues() if res.name not in ['HOH', 'WAT', 'NA', 'CL']]
    print(f"First 10 protein residues in B: {protein_residues[:10]}")
    

    # Randomly replace water molecules with Na and Cl ions
    totalIons = na_count + cl_count
    
    waterPos={}
    for res in modeller_b.topology.residues():
        if res.name == 'HOH':
            for atom in res.atoms():
                if atom.element == elem.oxygen:
                    waterPos[res] = modeller_b.positions[atom.index]
    
    solvated_ionized_topology, solvated_ionized_posiitons= addIons(modeller_b.topology, modeller_b.positions, waterPos, na_count, cl_count, ionCutoff=0.05*nanometer, positiveIon='Na+', negativeIon='Cl-')

    new_modeller_object = Modeller(solvated_ionized_topology, solvated_ionized_posiitons)
    
    # get the total number of atoms in new_modeller_object
    total_atoms = sum(1 for atom in new_modeller_object.topology.atoms())
    print(f"Total atoms in new_modeller_object after adding ions: {total_atoms}")


    # get the number of Na and Cl ions in the solvated conformation B
    na_count_b = sum(1 for res in new_modeller_object.topology.residues() if res.name == 'NA')
    cl_count_b = sum(1 for res in new_modeller_object.topology.residues() if res.name == 'CL')

    print(f"Na count in B: {na_count_b} and Cl count in B: {cl_count_b}")
    

    # Copy the topology from conformation A and the positions from the solvated conformation B to create the final Modeller object
    modeller_final = Modeller(prtn_a_topology, new_modeller_object.positions)

    # generate the system
    solvated_system = system_generator.create_system(modeller_final.topology)

    
    # check the charge of the system
    nonbonded_force = [f for f in solvated_system.getForces() if isinstance(f, NonbondedForce)][0]
    total_charge = sum(nonbonded_force.getParticleParameters(i)[0].value_in_unit(unit.elementary_charge) for i in range(nonbonded_force.getNumParticles()))
    print(f"Total charge of the system: {total_charge}")    

    

    # # Fix naked charges. We do not have to fix naked charges here. There will be a separate piece of code that takes care of this.
    # atom_map = {atom.index: atom.residue.name for atom in modeller_final.topology.atoms()}
    # force_dict = {force.__class__.__name__: force for force in solvated_system.getForces()}
    # if 'NonbondedForce' in [k for k in force_dict.keys()]:
    #     nb_force = force_dict['NonbondedForce']
    #     for idx in range(nb_force.getNumParticles()):
    #         if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
    #             continue
    #         charge, sigma, epsilon = nb_force.getParticleParameters(idx)
    #         if sigma == 0*unit.nanometer:
    #             new_sigma = 0.06*unit.nanometer
    #             nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
    #             print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
    #         if epsilon == 0*unit.kilojoule_per_mole:
    #             new_epsilon = 0.0001*unit.kilojoule_per_mole
    #             nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
    #             print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
    #             if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
    #                 new_sigma = 0.1*unit.nanometer
    #                 nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
    #                 print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

    
    # check the charge of the system after fixing naked charges
    nonbonded_force = [f for f in solvated_system.getForces() if isinstance(f, NonbondedForce)][0]
    total_charge = sum(nonbonded_force.getParticleParameters(i)[0].value_in_unit(unit.elementary_charge) for i in range(nonbonded_force.getNumParticles()))
    print(f"Total charge of the system after fixing naked charges: {total_charge}")

    return modeller_final, solvated_system

In [25]:
def get_residue_by_index(topology, residue_index):
    for residue in topology.residues():
        if residue.index == residue_index:
            return residue
    return None  # Return None if the residue is not found

In [35]:
# function to check if water atom is within a nm radius of protein
def check_water_proximity(topology, positions, water_residue_index:int, cutoff_nm:int=0.8*unit.nanometers):

    """

    Check if a water molecule is within a specified distance of the protein. Will be used to check if the alchemical water molecule is within the cutoff distance of the protein.
    Parameters
    ----------
    topology : simtk.openmm.app.Topology
        The topology object of the system.
    positions : simtk.unit.Quantity
        The positions of the atoms in the system.
    water_residue_index : int
        The index of the water residue in the topology.
    cutoff_nm : float, optional, default=0.8
        The cutoff distance in nanometers.
    Returns
    -------
    bool
        True if the water molecule is outside the cutoff distance, False otherwise.
    """



    import numpy as np

    
    # try to get residue by index
    water_residue = get_residue_by_index(topology, water_residue_index)
    if water_residue is None:
        raise ValueError(f"Residue index {water_residue_index} not found in the topology.")
    
    elif water_residue.name != 'HOH':
        raise ValueError(f"Residue index {water_residue_index} is not a water residue.")

    # Find the oxygen atom of the specified water residue
    water_oxygen_position = None
    for atom in water_residue.atoms():
        if atom.element == elem.oxygen:
            water_oxygen_position = positions[atom.index]
            break
        
    
    # If no matching water residue or oxygen atom is found, raise an error
    if water_oxygen_position is None:
        raise ValueError(f"Oxygen atom for water residue {water_residue_index} not found in topology.")
    
    # Calculate the distances between the water oxygen and all protein atoms
    for atom in topology.atoms():
        if atom.residue.name not in ['HOH','NA','CL']:  # Exclude water and ions
            protein_position = positions[atom.index]
            distance = np.linalg.norm(water_oxygen_position - protein_position)
            if distance <= cutoff_nm:
                return False  # Water is within cutoff distance
    
    return True  # Water is outside cutoff distance
    

In [42]:
# function to get a water molecule that is outside a nm radius of protein

def get_far_water_residue(charge_diff,
                           topology,
                           positions,
                           avoid_residues= [],
                           radius=0.8*unit.nanometers):
    """
    Uses the function check_water_proximity to find a water molecule that is outside a specified distance of the protein. returns a list of residue objects that are outside the specified radius. Length of the list is equal to the charge difference.

    Parameters
    ----------
    charge_diff : int
        The difference in the number of water molecules between the two systems.
    topology : simtk.openmm.app.Topology
        The topology object of the system.

    positions : simtk.unit.Quantity
        The positions of the atoms in the system.

    radius : float, optional, default=0.8
        The cutoff distance in nanometers.

    Returns
    -------
    list
        A list of residue objects that are outside the cutoff distance.

    """

    # Get the indices of all water residues
    water_residue_indices = [residue.index for residue in topology.residues() if residue.name == 'HOH']

    # Shuffle the water residue indices
    random.shuffle(water_residue_indices)

    # Initialize a list to store the water residues that are outside the cutoff distance
    far_water_residues = []

    # Iterate through the shuffled water residue indices
    for water_residue_index in water_residue_indices:
        if check_water_proximity(topology, positions, water_residue_index, cutoff_nm=radius):
            # get the residue object by index
            water_residue = get_residue_by_index(topology, water_residue_index)
            # check if the residue is not in the avoid_residues list
            if water_residue not in avoid_residues:
                far_water_residues.append(water_residue)
                if len(far_water_residues) == charge_diff:
                    break

    return far_water_residues


In [57]:
## function that takes two water residues, and swaps the positions of the oxygen and hydrogen atoms

def swap_water_atoms(water_residue_1, water_residue_2, positions):

    # check if the residues are water residues
    if water_residue_1.name != 'HOH' or water_residue_2.name != 'HOH':
        raise ValueError("At least one of the residues is not a water residues.")
    
    # exchange O atoms together, exchange H1 atoms together, exchange H2 atoms together

    # get the atom indices of the oxygen atoms
    water_1_oxygen_index = None
    water_2_oxygen_index = None
    for atom in water_residue_1.atoms():
        if atom.element == elem.oxygen:
            water_1_oxygen_index = atom.index
            break
    for atom in water_residue_2.atoms():
        if atom.element == elem.oxygen:
            water_2_oxygen_index = atom.index
            break
    
    # get the atom indices of the hydrogen atoms
    water_1_hydrogen_1_index = None
    water_1_hydrogen_2_index = None

    water_2_hydrogen_1_index = None
    water_2_hydrogen_2_index = None

    for atom in water_residue_1.atoms():
        if atom.element == elem.hydrogen and atom.name == 'H1':
            water_1_hydrogen_1_index = atom.index
        if atom.element == elem.hydrogen and atom.name == 'H2':
            water_1_hydrogen_2_index = atom.index

    for atom in water_residue_2.atoms():
        if atom.element == elem.hydrogen and atom.name == 'H1':
            water_2_hydrogen_1_index = atom.index
        if atom.element == elem.hydrogen and atom.name == 'H2':
            water_2_hydrogen_2_index = atom.index

    # swap the positions of the oxygen atoms

    temp = positions[water_1_oxygen_index]
    positions[water_1_oxygen_index] = positions[water_2_oxygen_index]
    positions[water_2_oxygen_index] = temp

    # swap the positions of the hydrogen atoms

    temp = positions[water_1_hydrogen_1_index]
    positions[water_1_hydrogen_1_index] = positions[water_2_hydrogen_1_index]
    positions[water_2_hydrogen_1_index] = temp

    temp = positions[water_1_hydrogen_2_index]

    positions[water_1_hydrogen_2_index] = positions[water_2_hydrogen_2_index]
    positions[water_2_hydrogen_2_index] = temp

    return positions



In [78]:
atom_ordering_dict = {
    'ALA': ['N', 'CA', 'C', 'O', 'CB', 'H', 'HA', 'HB1', 'HB2', 'HB3'],
    'ARG': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'NE', 'CZ', 'NH1', 'NH2', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HD2', 'HD3', 'HE', 'HH11', 'HH12', 'HH21', 'HH22'],
    'ASH': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'OD1', 'OD2', 'H', 'HA', 'HB2', 'HB3', 'HD2'],
    'ASN': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'OD1', 'ND2', 'H', 'HA', 'HB2', 'HB3', 'HD21', 'HD22'],#
    'ASP': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'OD1', 'OD2', 'H', 'HA', 'HB2', 'HB3'],
    'CYS': ['N', 'CA', 'C', 'O', 'CB', 'SG', 'H', 'HA', 'HB2', 'HB3','HG'], # this should work for both reduced and oxidized forms as it just checks the ordering, some pdbs might not be specific
    'GLH': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'OE1', 'OE2', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HE2'],
    'GLN': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'OE1', 'NE2', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HE21', 'HE22'],
    'GLU': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'OE1', 'OE2', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3'],
    'GLY': ['N', 'CA', 'C', 'O', 'H', 'HA2', 'HA3'],
    'HID': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'ND1', 'CD2', 'CE1', 'NE2', 'H', 'HA', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1'],
    'HIE': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'ND1', 'CD2', 'CE1', 'NE2', 'H', 'HA', 'HB2', 'HB3', 'HD2', 'HE1', 'HE2'],
    'HIP': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'ND1', 'CD2', 'CE1', 'NE2', 'H', 'HA', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2'],
    'HIS': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'ND1', 'CD2', 'CE1', 'NE2', 'H', 'HA', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2'],#His includes all the possible protonation states, as some pdbs might not be specific
    'ILE': ['N', 'CA', 'C', 'O', 'CB', 'CG1', 'CG2', 'CD1', 'H', 'HA', 'HB', 'HG12', 'HG13', 'HG21', 'HG22', 'HG23', 'HD11', 'HD12', 'HD13'],
    'LEU': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD1', 'CD2', 'H', 'HA', 'HB2', 'HB3', 'HG', 'HD11', 'HD12', 'HD13', 'HD21', 'HD22', 'HD23'],
    'LYN': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'CE', 'NZ', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HD2', 'HD3', 'HE2', 'HE3', 'HZ1', 'HZ2'],
    'LYS': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'CE', 'NZ', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HD2', 'HD3', 'HE2', 'HE3', 'HZ1', 'HZ2', 'HZ3'],
    'MET': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'SD', 'CE', 'H', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HE1', 'HE2', 'HE3'],
    'PHE': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'H', 'HA', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2', 'HZ'],
    'PRO': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD', 'HA', 'HB2', 'HB3', 'HG2', 'HG3', 'HD2', 'HD3'],
    'SER': ['N', 'CA', 'C', 'O', 'CB', 'OG', 'H', 'HA', 'HB2', 'HB3', 'HG'],
    'THR': ['N', 'CA', 'C', 'O', 'CB', 'OG1', 'CG2', 'H', 'HA', 'HB', 'HG1', 'HG21', 'HG22', 'HG23'],
    'TRP': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD1', 'CD2', 'NE1', 'CE2', 'CE3', 'CZ2', 'CZ3', 'CH2', 'H', 'HA', 'HB2', 'HB3', 'HD1', 'HE1', 'HE3', 'HZ2', 'HZ3', 'HH2'],
    'TYR': ['N', 'CA', 'C', 'O', 'CB', 'CG', 'CD1', 'CD2', 'CE1', 'CE2', 'CZ', 'OH', 'H', 'HA', 'HB2', 'HB3', 'HD1', 'HD2', 'HE1', 'HE2', 'HH'],
    'VAL': ['N', 'CA', 'C', 'O', 'CB', 'CG1', 'CG2', 'H', 'HA', 'HB', 'HG11', 'HG12', 'HG13', 'HG21', 'HG22', 'HG23'],
    # Add any additional non-standard residues 
    }

In [126]:
def update_conect_line(conect_line: str, atom_index_map: dict) -> str:
    """
    Updates the atom indices in the CONECT line based on the new atom_index_map.
    
    Parameters:
    conect_line : str
        Original CONECT line
    atom_index_map : dict
        Mapping of old atom indices to new atom indices
        
    Returns:
    updated_line : str
        Updated CONECT line with new atom indices
    """
    parts = conect_line.split()[1:]  # Skip 'CONECT'
    new_parts = []
    
    for atom_idx in parts:
        old_index = int(atom_idx)
        if old_index in atom_index_map:
            new_parts.append(str(atom_index_map[old_index]))
        else:
            new_parts.append(atom_idx)  # If not mapped, keep the original index

    return f"CONECT {' '.join(new_parts)}"

In [131]:
# function that takes a pdb file and rearranges the atoms in each residue to match the atom ordering in the atom_ordering_dict
# Only rearranges the atoms in the protein residues. Does not rearrange the atoms in water or ion residues.
# If an atom exists in the dict but not in the residue, it will not be added to the residue. This is importnat for residues such as cysteine that can exist in reduced and oxidized forms. the oxidized form is missing the hydrogen atom, and would thus not be added to the residue.
# If an atom exists in the residue but not in the dict, it raise an error.
# Function also keeps a map (dict) of the original atom indices to the new atom indices. Function then updates any CONECT records in the pdb file to reflect the new atom indices.

# Note that the names of the atoms in the pdb should already be according to the IUAPC naming convention. If not, the function will not work as expected. 


def order_pdb_by_convention(input_pdbfile:str, output_pdb_file:str, atom_ordering_dict:dict):
    
    # upload the pdb file and create a topology object  
    pdb = PDBFile(input_pdbfile)
    topology = pdb.topology
    positions = pdb.positions

    # create a dict to map the original atom indices to the new atom indices
    atom_index_map = dict()

    # create a modeller object
    modeller = Modeller(topology, positions)

    # get the residues in the modeller object
    residues = list(modeller.topology.residues())

    # iterate through the residues

    # create a new topology object
    new_topology = app.Topology()


    # Parse original PDB file for CONECT lines
    conect_lines = []
    with open(input_pdbfile, 'r') as pdb_file:
        for line in pdb_file:
            if line.startswith("CONECT"):
                conect_lines.append(line.strip())


    # iterate through the residues
    for residue in residues:

        # check the chain id of the residue
        residue_chain = residue.chain
        # check if the chain exists in the new topology object. If not, add it
        if residue_chain not in [chain.id for chain in new_topology.chains()]:
            new_chain = new_topology.addChain(residue_chain)

        #print(f"Residue name: {residue.name}")

        # check if residue is a protein residue

        if residue.name not in atom_ordering_dict.keys():
            # add the residue to the new topology object
            new_residue = new_topology.addResidue(residue.name, new_chain)
            for atom in residue.atoms():
                new_atom = new_topology.addAtom(atom.name, atom.element, new_residue)
                atom_index_map[atom.index] = new_atom.index
        else:

            # get the atom ordering for the residue
            atom_order = atom_ordering_dict[residue.name]

            # create a new residue in the new topology object
            new_residue = new_topology.addResidue(residue.name, new_chain)

            # create a dict to map the atom names to the atom objects
            atom_dict = {atom.name: atom for atom in residue.atoms()}

            # iterate through the atom_order list and add the atoms to the new residue
            for atom_name in atom_order:
                if atom_name in atom_dict.keys():
                    atom = atom_dict[atom_name]
                    new_atom = new_topology.addAtom(atom.name, atom.element, new_residue)
                    atom_index_map[atom.index] = new_atom.index
                else:
                    pass
            #check if all atoms in the residue are in the atom_order list
            for atom_name in atom_dict.keys():
                if atom_name not in atom_order:
                    raise ValueError(f"Atom {atom_name} in residue {residue.name} not found in atom_ordering_dict.")
            
            
        
    # Rearrange the positions of the atoms in the residue to match the atom ordering

    new_positions = [positions[atom_index_map[i]] for i in range(len(positions))]
    # extract the values from the positions into a new list
    new_position_values = [pos._value for pos in new_positions]

    # create a new modeller object
    new_modeller = Modeller(new_topology, new_positions)

    # prepare the CONECT lines to write as footer


    
    # write the new pdb file
    #TODO: Note that the writing of the pdb file positions follows  def _format_83(f) in openmm/app/pdbfile.py. This outputs a position with 3 decimal places, and the default unit is nanometers. If we want to add a 4th decimal place, we need to create a new function _format_84(f) in the pdbfile.py file.
    with open(output_pdb_file, 'w') as out_pdb:
        app.PDBFile.writeFile(new_modeller.topology, new_position_values, out_pdb, keepIds=True)
        
        
    # Delete the "END" Line and write the CONECT lines

    with open(output_pdb_file, 'r') as pdb_file:
        lines = pdb_file.readlines()
    with open(output_pdb_file, 'w') as pdb_file:
        for line in lines:
            if line.startswith("END"):
                continue
            pdb_file.write(line)
        for conect_line in conect_lines:
            updated_line = update_conect_line(conect_line, atom_index_map)
            pdb_file.write(updated_line + "\n")
        pdb_file.write("END\n")   


In [132]:
input_pdbfile = "/Users/Apple/Desktop/MD_Simulations/openmm_for perses/data_for_complete/apo-wt-1.pdb"
output_pdb_file = "/Users/Apple/Desktop/MD_Simulations/openmm_for perses/data_for_complete/apo-wt-1_reordered.pdb"

order_pdb_by_convention(input_pdbfile, output_pdb_file, atom_ordering_dict=atom_ordering_dict)

In [12]:
def minimize_system(topology: openmm.app.topology.Topology, positions: openmm.unit.quantity.Quantity, system: openmm.openmm.System, max_steps=1000):
    """
    Perform energy minimization after solvation to remove steric clashes.
    """
    max_steps = 1000
    temperature = 300*unit.kelvin
    collision_rate = 1/unit.picosecond
    timestep = 0.001*unit.picoseconds

    

    # Create a simulation context
    integrator = openmm.LangevinMiddleIntegrator(temperature, collision_rate, timestep)
    simulation = Simulation(topology, system, integrator)
    simulation.context.setPositions(positions)

    # Minimize the system
    print("Minimizing the system...")
    simulation.minimizeEnergy(maxIterations=max_steps)

    # Save the minimized structure
    positions = simulation.context.getState(getPositions=True).getPositions()
    return positions

In [300]:
#def minimize_system_2(topology: openmm.app.topology.Topology, positions: openmm.unit.quantity.Quantity, system: openmm.openmm.System, max_steps=1000):
    """
    Perform energy minimization after solvation to remove steric clashes.
    """
    import numpy as np


    max_steps = 1000
    temperature = 300*unit.kelvin
    collision_rate = 1/unit.picosecond
    timestep = 0.001*unit.picoseconds

    stages = [
        {'EOM': 'minimize', 'n_steps': 10000, 'temperature': 300*unit.kelvin, 'ensemble': None, 'restraint_selection': 'protein and not type H', 'force_constant': 100*unit.kilocalories_per_mole/unit.angstrom**2, 'collision_rate': 2/unit.picoseconds, 'timestep': 1*unit.femtoseconds},
        #{'EOM': 'MD_interpolate', 'n_steps': 100000, 'temperature': 100*unit.kelvin, 'temperature_end': 300*unit.kelvin, 'ensemble': 'NVT', 'restraint_selection': 'protein and not type H', 'force_constant': 100*unit.kilocalories_per_mole/unit.angstrom**2, 'collision_rate': 10/unit.picoseconds, 'timestep': 1*unit.femtoseconds}
    ]

    for i, parameters in enumerate(stages):
        
        
        
        
        # Make a copy of the system
        system_copy = copy.deepcopy(system)
        
        # Add restraint
        if parameters['restraint_selection'] is not None:

            # Format positions for mdtraj (should be a unitless array of arrays)
            formatted_positions = np.zeros(shape=(topology.getNumAtoms(), 3))
            for i, pos in enumerate(positions):
                formatted_positions[i] = np.array(pos.value_in_unit(unit.nanometers))

            traj = md.Trajectory(formatted_positions, md.Topology.from_openmm(topology))
            selection_indices = traj.topology.select(parameters['restraint_selection'])

            custom_cv_force = openmm.CustomCVForce('(K_RMSD/2)*(RMSD)^2')
            custom_cv_force.addGlobalParameter('K_RMSD', parameters['force_constant'] * 2)
            rmsd_force = openmm.RMSDForce(positions, selection_indices)
            custom_cv_force.addCollectiveVariable('RMSD', rmsd_force)
            system_copy.addForce(custom_cv_force)

        # Set barostat update interval to 0 (for NVT)
        if parameters['ensemble'] == 'NVT':
            force_dict = {force.__class__.__name__: index for index, force in enumerate(system_copy.getForces())}
            system_copy.removeForce(force_dict['MonteCarloBarostat']) # TODO : change this to `system_copy.getForce(force_dict['MonteCarloBarostat']).setFrequency(0)` once the next release comes out (this recently merged PR allows frequency to be 0: https://github.com/openmm/openmm/pull/3411) 
    
        elif parameters['ensemble'] == 'NPT' or parameters['ensemble'] is None:
            pass
        
        else:
            raise Exception("Invalid parameter supplied for 'ensemble'")
            
        # Set up integrator
        temperature = parameters['temperature']
        collision_rate = parameters['collision_rate']
        timestep = parameters['timestep']
        
        if parameters['EOM'] == 'MD_interpolate':
            temperature_end = parameters['temperature_end']
        
        integrator = openmm.LangevinMiddleIntegrator(temperature, collision_rate, timestep)
    
       
        
        sim = app.Simulation(topology, system_copy, integrator)
        sim.context.setPeriodicBoxVectors(*system_copy.getDefaultPeriodicBoxVectors())
        sim.context.setPositions(positions)
        sim.context.setVelocitiesToTemperature(temperature)
    


        # Run minimization or MD
        n_steps = parameters['n_steps']
        n_steps_per_iteration = 100
        
        if parameters['EOM'] == 'minimize':
            sim.minimizeEnergy(maxIterations=n_steps)



    


    # Save the minimized structure
    positions = sim.context.getState(getPositions=True).getPositions()
    return positions

In [ ]:
check_topology_alignment(wt_topology, protein_wt_p_topology)

In [13]:
modeller_object_p, system_p= solvate_with_conformation_a(prtn_a_topology=wt_topology, prtn_b_topology=protein_wt_p_topology, prtn_b_positions= protein_wt_p_positions, boxVectors=periodic_box_vectors)

protein_wt_p_topology_solvated, protein_wt_p_positions_solvated = modeller_object_p.topology, modeller_object_p.positions

protein_wt_p_md_topology_solvated = md.Topology.from_openmm(protein_wt_p_topology_solvated)

Water count in A: 7748, Na count in A: 7, Cl count in A: 9
Total number of atoms in A: 24969
Total number of protein residues in A: 110
Original Na count in B: 0 and Original Cl count in B: 0
Water count in B: 7765 and total solvent count: 7764
Removing water molecules from B
Water count in B after adding or removing: 7764 and total solvent count: 7764
Total atoms in modeller_b: 25001
Na count in B: 0 and Cl count in B: 0
First 10 protein residues in B: [<Residue 0 (ACE) of chain 0>, <Residue 1 (VAL) of chain 0>, <Residue 2 (ILE) of chain 0>, <Residue 3 (ASN) of chain 0>, <Residue 4 (THR) of chain 0>, <Residue 5 (PHE) of chain 0>, <Residue 6 (ASP) of chain 0>, <Residue 7 (GLY) of chain 0>, <Residue 8 (VAL) of chain 0>, <Residue 9 (ALA) of chain 0>]
pickedMol: <Residue 1380 (HOH) of chain 1>
pickedMol: <Residue 3486 (HOH) of chain 1>
pickedMol: <Residue 6272 (HOH) of chain 1>
pickedMol: <Residue 4334 (HOH) of chain 1>
pickedMol: <Residue 3548 (HOH) of chain 1>
pickedMol: <Residue 3654 (

In [37]:
check_water_proximity(protein_wt_p_topology_solvated, protein_wt_p_positions_solvated, 6278)

True

In [61]:
aa_list= get_far_water_residue(1, protein_wt_p_topology_solvated, protein_wt_p_positions_solvated)

residue_1= aa_list[0]
print(residue_1)


# print the atom positions of all atoms in the residue
for atom in residue_1.atoms():
    print(atom.index, atom.name, protein_wt_p_positions_solvated[atom.index])

<Residue 4770 (HOH) of chain 2>
15689 O Vec3(x=2.562859535217285, y=2.080153465270996, z=2.157008171081543) nm
15690 H1 Vec3(x=2.477694034576416, y=2.0458130836486816, z=2.1840245723724365) nm
15691 H2 Vec3(x=2.626063108444214, y=2.032961845397949, z=2.211235284805298) nm


In [62]:
aa_list= get_far_water_residue(1, protein_wt_p_topology_solvated, protein_wt_p_positions_solvated)

residue_2= aa_list[0]
print(residue_2)

# print the atom positions of all atoms in the residue
for atom in residue_2.atoms():
    print(atom.index, atom.name, protein_wt_p_positions_solvated[atom.index])

<Residue 4520 (HOH) of chain 2>
14939 O Vec3(x=4.695365905761719, y=3.562401294708252, z=-1.834792137145996) nm
14940 H1 Vec3(x=4.67686653137207, y=3.480325698852539, z=-1.78914475440979) nm
14941 H2 Vec3(x=4.674035549163818, y=3.5442874431610107, z=-1.9263302087783813) nm


In [63]:
swapped_positions= swap_water_atoms(residue_1, residue_2, protein_wt_p_positions_solvated)

print("After swapping the water molecules")

print(residue_1)
for atom in residue_1.atoms():
    print(atom.index, atom.name, protein_wt_p_positions_solvated[atom.index])

print(residue_2)
for atom in residue_2.atoms():
    print(atom.index, atom.name, protein_wt_p_positions_solvated[atom.index])

After swapping the water molecules
<Residue 4770 (HOH) of chain 2>
15689 O Vec3(x=4.695365905761719, y=3.562401294708252, z=-1.834792137145996) nm
15690 H1 Vec3(x=4.67686653137207, y=3.480325698852539, z=-1.78914475440979) nm
15691 H2 Vec3(x=4.674035549163818, y=3.5442874431610107, z=-1.9263302087783813) nm
<Residue 4520 (HOH) of chain 2>
14939 O Vec3(x=2.562859535217285, y=2.080153465270996, z=2.157008171081543) nm
14940 H1 Vec3(x=2.477694034576416, y=2.0458130836486816, z=2.1840245723724365) nm
14941 H2 Vec3(x=2.626063108444214, y=2.032961845397949, z=2.211235284805298) nm


In [52]:
# get the number of atoms in the solvated protein topology
n_atoms_solvated = protein_wt_p_topology_solvated.getNumAtoms()
# get the number of atoms in the solvated protein positions
n_atoms_solvated_positions = len(protein_wt_p_positions_solvated)

print(f"Number of atoms in the solvated protein topology: {n_atoms_solvated}")
print(f"Number of atoms in the solvated protein positions: {n_atoms_solvated_positions}")

n_atoms_solvated_wt = wt_topology.getNumAtoms()
print(f"Number of atoms in the solvated wt protein topology: {n_atoms_solvated_wt}")

Number of atoms in the solvated protein topology: 24969
Number of atoms in the solvated protein positions: 24969
Number of atoms in the solvated wt protein topology: 24969


In [186]:
# print the protein_wt_p_positions_solvated to a txt file
with open('protein_wt_p_positions_solvated.txt', 'w') as f:
    for item in protein_wt_p_positions_solvated:
        f.write("%s\n" % item)

In [15]:

protein_wt_p_positions_solvated =minimize_system(protein_wt_p_topology_solvated, protein_wt_p_positions_solvated, system_p)



Minimizing the system...


In [ ]:
## We will now handle the problem of creating a hybrid topology and hybrid positions for both the wt and mutant conformations. The hybrid topology of the both the wt and mutant proteins should be identical. We have now generated the solvated positions for all conformations. We will have to create the hybrid positions by adding the missing atoms of the missing residue, while making sure that the positions make sense. We cannot just add the positions of whatever is generated in the original htf because the positions might not make physical sense.